In [1]:
import torch

In [2]:
!pip install pandas

  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.5 MB 2.4 MB/s eta 0:00:05
   --- ------------------------------------ 1.0/11.5 MB 2.1 MB/s eta 0:00:05
   ---- ----------------------------------- 1.3/11.5 MB 2.0 MB/s eta 0:00:06
   ------ --------------------------------- 1.8/11.5 MB 2.0 MB/s eta 0:00:05
   ------- -------------------------------- 2.1/11.5 MB 1.8 MB/s eta 0:00:06
   -------- ------------------------------- 2.4/11.5 MB 1.8 MB/s eta 0:00:05
   ---------- ----------------------------- 2.9/11.5 MB 1.9 MB/s eta 0:00:05
   ---------- ----------------------------- 3.1/11.5 MB 1.8 MB/s eta 0:00:05
   ----------- ---------------------------- 3.4/11.5 MB 1.7 MB/s eta 0:00:05
   ----------- -------------

In [3]:
import pandas as pd
import numpy as np

In [4]:
data=pd.read_csv("data.csv")

In [8]:
# Load the dataset again with the correct delimiter
data = pd.read_csv("data.csv", delimiter=';')

# Display the first few rows of the dataset
data.head()


,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [9]:
# Define features and target
X = data.drop('Target', axis=1)  # Features (independent variables)
y = data['Target']                # Target variable (dependent variable)

# Check the shapes of X and y
X_shape = X.shape
y_shape = y.shape

X_shape, y_shape


((4424, 36), (4424,))

In [12]:
import pandas as pd
import numpy as np
import torch

# Load the dataset with the correct delimiter
data = pd.read_csv('data.csv', delimiter=';')

# Check data types
print(data.dtypes)

# Convert categorical columns to numeric (label encoding)
categorical_columns = [
    'Marital status',
    'Application mode',
    'Course',
    'Daytime/evening attendance\t',
    'Previous qualification',
    'Nacionality',
    "Mother's qualification",
    "Father's qualification",
    "Mother's occupation",
    "Father's occupation",
    'Displaced',
    'Educational special needs',
    'Debtor',
    'Tuition fees up to date',
    'Gender',
    'Scholarship holder',
    'International'
]

# Convert categorical columns to category codes
for col in categorical_columns:
    if col in data.columns:
        data[col] = data[col].astype('category').cat.codes

# Define features and target
X = data.drop('Target', axis=1)  # Features (independent variables)
y = data['Target']                # Target variable (dependent variable)

# Convert X to numeric, forcing errors to NaN (and handle NaNs)
X = X.apply(pd.to_numeric, errors='coerce')

# Convert y to numeric, forcing errors to NaN (and handle NaNs)
y = pd.to_numeric(y, errors='coerce')

# Check for NaNs in X and y and drop them
X = X.dropna()
y = y[X.index]  # Align y with the new X after dropping rows

# Convert to NumPy arrays
X = X.values
y = y.values

# Convert to tensors
X_tensor = torch.tensor(X, dtype=torch.float32)  # For features
y_tensor = torch.tensor(y, dtype=torch.float32)  # For target (or torch.long for classification tasks)

# Check the shapes
print("Features tensor shape:", X_tensor.shape)
print("Target tensor shape:", y_tensor.shape)


Marital status                                      int64
Application mode                                    int64
Application order                                   int64
Course                                              int64
Daytime/evening attendance\t                        int64
Previous qualification                              int64
Previous qualification (grade)                    float64
Nacionality                                         int64
Mother's qualification                              int64
Father's qualification                              int64
Mother's occupation                                 int64
Father's occupation                                 int64
Admission grade                                   float64
Displaced                                           int64
Educational special needs                           int64
Debtor                                              int64
Tuition fees up to date                             int64
Gender        

In [16]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, csv_file):
        # Load the dataset
        self.data = pd.read_csv(csv_file, delimiter=';')  # Adjust delimiter if needed
        
        # Convert categorical columns to numeric (label encoding)
        categorical_columns = [
            'Marital status',
            'Application mode',
            'Course',
            'Daytime/evening attendance\t',
            'Previous qualification',
            'Nacionality',
            "Mother's qualification",
            "Father's qualification",
            "Mother's occupation",
            "Father's occupation",
            'Displaced',
            'Educational special needs',
            'Debtor',
            'Tuition fees up to date',
            'Gender',
            'Scholarship holder',
            'International'
        ]

        for col in categorical_columns:
            if col in self.data.columns:
                self.data[col] = self.data[col].astype('category').cat.codes
        
        # Define features and target
        self.X = self.data.drop('Target', axis=1)  # Features (independent variables)
        self.y = self.data['Target']                # Target variable (dependent variable)

        # Convert X to numeric, forcing errors to NaN (and handle NaNs)
        self.X = self.X.apply(pd.to_numeric, errors='coerce')

        # Convert y to numeric, forcing errors to NaN (and handle NaNs)
        self.y = pd.to_numeric(self.y, errors='coerce')

        # Drop rows with NaN values from both X and y to ensure alignment
        self.data = self.data.dropna(subset=['Target'])  # Drop NaNs in target
        self.X = self.X.loc[self.data.index]  # Align X with cleaned data
        self.y = self.y.loc[self.data.index]  # Align y with cleaned data

    def __len__(self):
        return len(self.y)  # Return the length of the target array

    def __getitem__(self, idx):
        x_sample = torch.tensor(self.X.iloc[idx].values, dtype=torch.float32)  # Convert feature to tensor
        y_sample = torch.tensor(self.y.iloc[idx], dtype=torch.float32)  # Convert target to tensor
        return x_sample, y_sample

# Example usage
if __name__ == "__main__":
    # Create an instance of the dataset
    dataset = CustomDataset('data.csv')
    
    # Print the length of the dataset
    print(f"Dataset size: {len(dataset)}")
    
    # Create a DataLoader
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    # Iterate through the DataLoader and print a batch
    for x_batch, y_batch in dataloader:
        print("X batch:", x_batch)
        print("Y batch:", y_batch)
        break  # Remove this line if you want to see more batches


Dataset size: 4424
X batch: tensor([[ 0.0000,  0.0000,  2.0000,  ..., 16.2000,  0.3000, -0.9200],
        [ 0.0000,  0.0000,  2.0000,  ..., 16.2000,  0.3000, -0.9200],
        [ 0.0000, 16.0000,  1.0000,  ...,  8.9000,  1.4000,  3.5100],
        ...,
        [ 0.0000,  7.0000,  1.0000,  ..., 11.1000,  0.6000,  2.0200],
        [ 0.0000, 11.0000,  1.0000,  ..., 10.8000,  1.4000,  1.7400],
        [ 0.0000,  0.0000,  5.0000,  ..., 10.8000,  1.4000,  1.7400]])
Y batch: tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan])
